In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, callbacks
from sklearn.metrics import roc_auc_score

In [2]:
read_dir = './pipeline/cleaned/'

cohort = pd.read_csv(read_dir + 'cohort.csv', index_col=0)

diag = pd.read_csv(read_dir + 'diag.csv', index_col=0)
diag = diag.set_index('hadm_id')

labs = pd.read_csv(read_dir + 'labs.csv', index_col=0)
labs = labs.set_index('hadm_id')

proc = pd.read_csv(read_dir + 'proc.csv', index_col=0)
proc = proc.set_index('hadm_id')

meds = pd.read_csv(read_dir + 'meds.csv', index_col=0)
meds = meds.set_index('hadm_id')

train_hadm = cohort[cohort.split=='train'].hadm_id.to_numpy()
valid_hadm = cohort[cohort.split=='validation'].hadm_id.to_numpy()
test_hadm = cohort[cohort.split=='test'].hadm_id.to_numpy()
test_hadm = np.sort(test_hadm)

In [6]:
class Transformer(object):
    def __init__(self, lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf):
        keras.backend.clear_session()
        physical_devices = tf.config.experimental.list_physical_devices('GPU')
        #print(f"GPU list {physical_devices}")
        #tf.config.experimental.set_memory_growth(physical_devices[0], True)
        self.opt = keras.optimizers.Adam(learning_rate=lr)
        self.loss = keras.losses.BinaryCrossentropy()
        self.metric = keras.metrics.AUC(name='auc')
        
        self.cohort = cohort
        self.diag = diag.copy()
        self.labs = labs
        self.proc = proc
        self.meds = meds
        
        self.use_diag = use_diag
        self.use_labs = use_labs
        self.use_proc = use_proc
        self.use_meds = use_meds
        self.use_insurance = use_insurance
        self.use_gender = use_gender
        self.use_race = use_race
        
        self.input_shapes = self.get_input_shapes()
        if hf:
            self.label = 'label_hf'
            self.diag['I50'] = 0
        else:
            self.label = 'label_diabetes'
            self.diag['E11'] = 0
            
        self.buildModel()
    
    def embedding(self, inputs):
        maxlen = inputs.shape[-2]
        varnum = inputs.shape[-1]
        pos = keras.backend.arange(start=0, stop=maxlen, step=1)
        pos = layers.Embedding(input_dim=maxlen, output_dim=256)(pos)
        x = layers.Dense(units=256)(inputs)
        return x + pos
    
    def encoder(self, inputs):
        hidden_dim = inputs.shape[-1]
        if len(inputs.shape) == 2:
            y = layers.Dense(hidden_dim)(inputs)
            y = layers.Dropout(0.1)(y)
            y = layers.LayerNormalization(epsilon=1e-6)(y)
            return tf.expand_dims(y, 1)
        
        x = layers.MultiHeadAttention(num_heads=8, key_dim=hidden_dim)(inputs, inputs)
        x = layers.Dropout(0.1)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
        
        y = layers.Dense(hidden_dim, activation='relu')(x)
        y = layers.Dense(hidden_dim)(y)
        y = layers.Dropout(0.1)(y)
        y = layers.LayerNormalization(epsilon=1e-6)(y + x)
        return y
    
    def buildModel(self): 
        inputs = [keras.Input(shape=shape) for shape in self.input_shapes]
        encoded = [self.encoder(inp) for inp in inputs]
        if len(inputs) > 1:
            combined = layers.Concatenate(axis=-1)([layers.GlobalAveragePooling1D()(enc) for enc in encoded])
        else:
            combined = layers.GlobalAveragePooling1D()(encoded[0])
        x = layers.Dense(64, activation="relu")(combined)
        outputs = layers.Dense(1, activation="sigmoid")(x)

        self.model = keras.Model(inputs=inputs, outputs=outputs)
        
        self.model.compile(optimizer=self.opt, loss=self.loss, metrics=[self.metric])
    
    def train(self, train_hadm, valid_hadm, epochs, batchSize, path, val_auc=False, verbose_ckp=1, verbose_fit=2):
        if val_auc:
            MCP = callbacks.ModelCheckpoint(path, monitor='val_auc', verbose=verbose_ckp, save_best_only=True, mode='max')
            ES = callbacks.EarlyStopping(monitor='val_auc', patience=10, verbose=verbose_ckp, mode='max')
        else:
            MCP = callbacks.ModelCheckpoint(path, monitor='val_loss', verbose=verbose_ckp, save_best_only=True, mode='min') 
            ES = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_ckp, mode='min')
        train_gen = self.generator(train_hadm, batchSize)
        valid_gen = self.generator(valid_hadm, batchSize)
        self.model.fit(train_gen, validation_data=valid_gen, epochs=epochs, verbose=verbose_fit, callbacks=[MCP, ES], 
                       steps_per_epoch=len(train_hadm)//batchSize, validation_steps=len(valid_hadm)//batchSize)
        self.model = models.load_model(path, compile=False)

    def load(self, path):
        self.model = models.load_model(path, compile=False)
        
    def predict(self, test_hadm, batchSize, verbose=2):
        pred_gen = self.pred_generator(test_hadm, batchSize)
        pred = self.model.predict(pred_gen, verbose=verbose, steps=(len(test_hadm)+batchSize-1)//batchSize)
        return pred

    def generator(self, input_hadm, batchSize):
        while 1:
            hadm_ids = np.sort(np.random.choice(input_hadm, batchSize, False))
            x = self.get_x(hadm_ids, batchSize)
            y = self.get_y(hadm_ids)
            yield x, y
            
    def pred_generator(self, input_hadm, batchSize):
        i = 0
        while 1:
            hadm_ids = input_hadm[np.arange(i, min(i+batchSize, len(input_hadm)))]
            if i+batchSize >= len(input_hadm):
                i = 0
            i += len(hadm_ids)
            x = self.get_x(hadm_ids, len(hadm_ids))
            y = self.get_y(hadm_ids)
            yield x, y
            
    def get_y(self, hadm_ids):
        batch_cohort = self.cohort[self.cohort.hadm_id.isin(hadm_ids)].sort_values('hadm_id')
        y = batch_cohort[self.label].to_numpy()
        return y

    def get_input_shapes(self):
        xs = []
        xd = ( 3, 306)
        xl = (14, 107)
        xp = ( 2,  18)
        xm = (14,  55)
    
        if self.use_diag:
            xs.append(xd)

        if self.use_labs:
            xs.append(xl)

        if self.use_proc:
            xs.append(xp)
            
        if self.use_meds:
            xs.append(xm)
            
        cohort_columns = []
        if self.use_insurance:
            cohort_columns.append('medicare')
            cohort_columns.append('medicaid')
        if self.use_gender:
            cohort_columns.append('male')
        if self.use_race:
            cohort_columns.append('white')
        if len(cohort_columns) > 0:
            xs.append((len(cohort_columns),))

        return xs
    
    def get_x(self, hadm_ids, batchSize):
        batch_cohort = self.cohort[self.cohort.hadm_id.isin(hadm_ids)].sort_values('hadm_id')

        xs = []
        xd = np.zeros((batchSize,  3, 306))
        xl = np.zeros((batchSize, 14, 107))
        xp = np.zeros((batchSize,  2,  18))
        xm = np.zeros((batchSize, 14,  55))

        if self.use_diag:
            for i in range(batchSize):
                seqnum = min(xd.shape[1], batch_cohort.seqnum.iloc[i]+1)
                raw_index = batch_cohort.index[i]
                xd[i, 0:seqnum] = self.diag.loc[self.cohort[(raw_index-seqnum+1):(raw_index+1)].hadm_id.to_numpy()].to_numpy()
            xs.append(xd)

        if self.use_labs:
            for i in range(batchSize):
                hadm_id = hadm_ids[i]
                if hadm_id in self.labs.index:
                    labs_i = self.labs.loc[hadm_id:hadm_id+1]
                    seqnum = min(xl.shape[1], len(labs_i))
                    xl[i, 0:seqnum] = labs_i.iloc[-seqnum:].to_numpy()
            xs.append(xl)

        if self.use_proc:
            for i in range(batchSize):
                hadm_id = hadm_ids[i]
                if hadm_id in self.proc.index:
                    proc_i = self.proc.loc[hadm_id:hadm_id+1]
                    seqnum = min(xp.shape[1], len(proc_i))
                    xp[i, 0:seqnum] = proc_i.iloc[-seqnum:].to_numpy()
            xs.append(xp)

        if self.use_meds:
            for i in range(batchSize):
                hadm_id = hadm_ids[i]
                if hadm_id in self.meds.index:
                    meds_i = self.meds.loc[hadm_id:hadm_id+1]
                    seqnum = min(xm.shape[1], len(meds_i))
                    xm[i, 0:seqnum] = meds_i.iloc[-seqnum:].to_numpy()
            xs.append(xm)

        cohort_columns = []
        if self.use_insurance:
            cohort_columns.append('medicare')
            cohort_columns.append('medicaid')
        if self.use_gender:
            cohort_columns.append('male')
        if self.use_race:
            cohort_columns.append('white')
        if len(cohort_columns) > 0:
            xs.append(batch_cohort[cohort_columns].to_numpy())

        return xs

# Labs

In [7]:
use_diag = 0
use_labs = 1
use_proc = 0
use_meds = 0
use_insurance = 0
use_gender = 0
use_race = 0
hf = 1
lr = 1e-3
epochs = 100
batchSize = 256
path='./models/labs.h5'

model = Transformer(lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf)
model.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 14, 107)]    0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 14, 107)     369043      ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 14, 107)      0           ['multi_head_attention[0][0]']   
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 14, 107)     0           ['dropout[0][0]',            

In [8]:
model = Transformer(lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf)
model.train(train_hadm, valid_hadm, epochs, batchSize, path, False, 1, 1)

Epoch 1/100
  1/254 [..............................] - ETA: 8:34 - loss: 0.8983 - auc: 0.5847

2024-03-28 22:56:17.945626: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


254/254 [==============================] - ETA: 0s - loss: 0.3207 - auc: 0.5598
Epoch 00001: val_loss improved from inf to 0.30101, saving model to ./models/labs.h5
254/254 [==============================] - 73s 282ms/step - loss: 0.3207 - auc: 0.5598 - val_loss: 0.3010 - val_auc: 0.7021
Epoch 2/100
254/254 [==============================] - ETA: 0s - loss: 0.3110 - auc: 0.5910
Epoch 00002: val_loss did not improve from 0.30101
254/254 [==============================] - 71s 280ms/step - loss: 0.3110 - auc: 0.5910 - val_loss: 0.3104 - val_auc: 0.7512
Epoch 3/100
254/254 [==============================] - ETA: 0s - loss: 0.3143 - auc: 0.5956
Epoch 00003: val_loss did not improve from 0.30101
254/254 [==============================] - 71s 280ms/step - loss: 0.3143 - auc: 0.5956 - val_loss: 0.3012 - val_auc: 0.7674
Epoch 4/100
254/254 [==============================] - ETA: 0s - loss: 0.3174 - auc: 0.6069
Epoch 00004: val_loss did not improve from 0.30101
254/254 [=========================

In [9]:
model.load(path)
res = model.predict(test_hadm, batchSize, 1)
y_true = cohort.set_index('hadm_id').loc[test_hadm].label_hf.to_numpy()
roc_auc_score(y_true, res)

86/86 [==============================] - 17s 202ms/step


0.810232935296863

# Diag

In [10]:
use_diag = 1
use_labs = 0
use_proc = 0
use_meds = 0
use_insurance = 0
use_gender = 0
use_race = 0
hf = 1
lr = 1e-3
epochs = 100
batchSize = 256
path='./models/diag.h5'

model = Transformer(lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf)
model.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 306)]     0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 3, 306)      3004002     ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 3, 306)       0           ['multi_head_attention[0][0]']   
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 3, 306)      0           ['dropout[0][0]',            

In [11]:
model = Transformer(lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf)
model.train(train_hadm, valid_hadm, epochs, batchSize, path, False, 1, 1)

Epoch 1/100
254/254 [==============================] - ETA: 0s - loss: 0.3025 - auc: 0.6660
Epoch 00001: val_loss improved from inf to 0.25810, saving model to ./models/diag.h5
254/254 [==============================] - 40s 152ms/step - loss: 0.3025 - auc: 0.6660 - val_loss: 0.2581 - val_auc: 0.8905
Epoch 2/100
254/254 [==============================] - ETA: 0s - loss: 0.2843 - auc: 0.7310
Epoch 00002: val_loss improved from 0.25810 to 0.24600, saving model to ./models/diag.h5
254/254 [==============================] - 38s 152ms/step - loss: 0.2843 - auc: 0.7310 - val_loss: 0.2460 - val_auc: 0.8912
Epoch 3/100
254/254 [==============================] - ETA: 0s - loss: 0.2796 - auc: 0.7407
Epoch 00003: val_loss did not improve from 0.24600
254/254 [==============================] - 38s 151ms/step - loss: 0.2796 - auc: 0.7407 - val_loss: 0.2502 - val_auc: 0.8874
Epoch 4/100
254/254 [==============================] - ETA: 0s - loss: 0.2746 - auc: 0.7621
Epoch 00004: val_loss did not impro

In [12]:
model.load(path)
res = model.predict(test_hadm, batchSize, 1)
y_true = cohort.set_index('hadm_id').loc[test_hadm].label_hf.to_numpy()
roc_auc_score(y_true, res)

86/86 [==============================] - 9s 106ms/step


0.8880655034863912

# All

In [13]:
use_diag = 1
use_labs = 1
use_proc = 1
use_meds = 1
use_insurance = 1
use_gender = 1
use_race = 1
hf = 1
lr = 1e-3
epochs = 100
batchSize = 256
path='./models/all.h5'

model = Transformer(lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf)
model.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 306)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 14, 107)]    0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 2, 18)]      0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 14, 55)]     0           []                               
                                                                                              

In [14]:
model = Transformer(lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf)
model.train(train_hadm, valid_hadm, epochs, batchSize, path, False, 1, 1)

Epoch 1/100
254/254 [==============================] - ETA: 0s - loss: 0.3052 - auc: 0.6864
Epoch 00001: val_loss improved from inf to 0.26751, saving model to ./models/all.h5
254/254 [==============================] - 120s 462ms/step - loss: 0.3052 - auc: 0.6864 - val_loss: 0.2675 - val_auc: 0.8760
Epoch 2/100
254/254 [==============================] - ETA: 0s - loss: 0.2821 - auc: 0.7404
Epoch 00002: val_loss did not improve from 0.26751
254/254 [==============================] - 115s 455ms/step - loss: 0.2821 - auc: 0.7404 - val_loss: 0.2800 - val_auc: 0.8492
Epoch 3/100
254/254 [==============================] - ETA: 0s - loss: 0.2784 - auc: 0.7517
Epoch 00003: val_loss improved from 0.26751 to 0.24858, saving model to ./models/all.h5
254/254 [==============================] - 115s 456ms/step - loss: 0.2784 - auc: 0.7517 - val_loss: 0.2486 - val_auc: 0.8901
Epoch 4/100
254/254 [==============================] - ETA: 0s - loss: 0.2696 - auc: 0.7692
Epoch 00004: val_loss did not impr

In [15]:
model.load(path)
res = model.predict(test_hadm, batchSize, 1)
y_true = cohort.set_index('hadm_id').loc[test_hadm].label_hf.to_numpy()
roc_auc_score(y_true, res)

86/86 [==============================] - 28s 323ms/step


0.84768888185124